# 说明

为什么要用卷积神经网络：

图像分类，用多层全连接的ANN肯定是可以的！但是因为中间都是Dense层，所以输入的图片必须要Flatten展平为一维向量后，作为后面Dense层的输入：每张图片有多少像素，就意味着输入层有多少个特征！即无法将一张图片当成一个整体来看，必须当成一个个组成它的像素点！—— 小图（28x28=784）输入层784个特征还能接受，但当每张图片是2000x2000 = 400w时，再将图片按照一个个像素点来对待会极大增加网络容量！

总结：ANN把图片按照一个个组成它的像素点来对待是**非常合理的**，因为图像里所有的信息就在于那些像素值！但是如此对待虽然最为精确，但是网络的训练负担太重了！因此CNN的出现就是以牺牲部分精确度为代价（深层抽象，甚至是不牺牲），来极大降低网络的可训练性！—— 所以：卷积神经网络CNN就是来降低ANN网络的网络容量的，使大图数据集变得可以处理！

注意：
- CNN是如何降低训练负担的：不断下采样，图像不断变小（舍弃不重要信息）的同时获取其抽象特征！—— 此外：CNN其实没有ANN中传统的“神经元”的概念，取而代之的是“卷积核”，即ANN训练的是神经元间的权重和偏置，现在CNN就只用训练各个卷积核中的数值而已，大大减少需要训练的参数！
- CNN网络离不开Dense层！因为已知全连接层所包含的信息是最完备的（什么都不丢弃）；所以当前面各种卷积、池化操作之后，最后还是要用Dense层把各种有用信息“**无丢弃**”的使用起来（汇总作为最后分类的依据）。
- 感想1：**全连接层是最好的**！它没有丢弃任何有效的信息！但它的缺点也十分明显：需要太多的神经元，太大的网络容量！—— CNN尽量提取最有效区分彼此的“大特征”来减少网络的容量，但并非完全不用Dense层！
- 感谢2：下采样虽然看上去只是在缩小图片大小，但其实相对应也增大了同尺寸卷积核的感受野（在小范围内其相对变大）！

CNN网络搭建：
- 卷积层：conv2d + 激活函数relu/sigmoid/tanh —— 纯粹是为了提取有效信息！
- 池化层：pooling2d —— 降采样：纯粹是为了将图片变小，丢弃非主要信息（可认为是噪声，即提高信噪比！）！
- 重复上面两种层的搭建
- 全连接层：Dense —— 最后对已提取的有效信息的汇总

卷积层关键参数：
- filters：用多少个卷积核 —— 每个卷积核提取一种特征（运算后生成一张图像），即该卷积层要提取多少个特征！关键！
- ksize：卷积核大小
- strides：卷积核移动的跨度；若跨度大于1，则卷积层后的图像会变小！
- padding：是否边缘填充0；若不填valid(same)，卷积层后图也会稍微变小！
- activation：卷积计算后是否再走一次激活
- use_bias：卷积计算后是否加一个偏置
- **注意：CNN中除全连接层外，所有的待训练参数都来自卷积层的卷积核里面的数值以及偏置！**

池化层关键参数：因为有卷积层的特征提取，也就是有了池化层丢弃无用小特征！—— 最大值池化（常用）；平均池化
- ksize：池化核大小
- strides：池化核移动的跨度
- padding：同上
- **注意：池化层没有filters参数，没有需要训练的参数！!**所以前面一层（一般都是卷积层）输出的图像个数（即特征个数），就是池化层的池化核个数！例如：前一卷积层提取4个特征，即生成了4张图像；传入到池化层后，该池化层就有4个池化核，一一对应的对4张图像进行下采样！—— 可以看出：池化层就是非常简单的对传入图像进行缩小而已！！

对CNN整个流程的说明：以传入一张64x64图像为例 —— 不断变厚（特征提取，通道数变多）、变小（下采样）
- 卷积层1：3个卷积核、卷积核大小(3,3)、移动步长1、padding为same —— 输入：1张64x64的图像；**输出**：每个卷积核提取了一个特征，故共提取了3个特征，即新生成了3个64x64的特征图像（用一个高维数组记录(64,64,3)）；
- 池化层1：3个池化核（已根据输入内定），池化核大小(2,2)、移动步长1、padding为same —— 输入：3张64x64的图像(64,64,3)；**输出**：对每张图像进行下采样，生成3张变小了的图像（也用一个高维数组记录(32,32,3)）；
- 卷积层2：4个卷积核、卷积核大小(3,3)、移动步长1、padding为same —— 输入：3张32x32图像(32,32,3)；**输出**：每个卷积核对输入进行特征提取，故共提取4个特征，即新生成了4个32x32的特征图像（用一个高维数组记录(32,32,4)）—— 这4个特征图是怎么来的？答：每个特征图并不是完全由所有的输入得来的！有的特征图是由输入中某几个特征图给它的！[参考这篇](https://www.cnblogs.com/CJT-blog/p/10562819.html)
- 池化层2：4个池化核（已根据输入内定），池化核大小(2,2)、移动步长1、padding为same —— 输入：4张32x32的图像(32,32,4)；**输出**：对每张图像进行下采样，生成4张变小了的图像（也用一个高维数组记录(16,16,4)）；
- 展平层：把输入的每张图像(16,16)展平为一维向量，为输入到全连接层做准备！
- 全连接层：神经元数为256（可自定）
- 输出层：10个神经元（做10分类）

下面用MINST数据集来展示CNN流程中图像的变化：model.output_shape查看每层输出情况！

In [3]:
import tensorflow as tf

In [4]:
(train_image, train_label), (test_image, test_label) = tf.keras.datasets.mnist.load_data()
train_image.shape

(60000, 28, 28)

In [5]:
model = tf.keras.Sequential()

In [6]:
# 卷积层1输出：3个卷积核，就生成了3张图片，记为(28,28,3)
model.add( tf.keras.layers.Conv2D(3, (3,3), input_shape=(28,28,1), activation = 'relu', padding = 'same') )
model.output_shape

(None, 28, 28, 3)

In [7]:
# 池化层1输出：已内定为3个池化核，仅仅是对输入3张图的缩小一半（用默认池化核大小）而已
model.add( tf.keras.layers.MaxPooling2D() )
model.output_shape

(None, 14, 14, 3)

In [8]:
# 卷积层2输出：4个卷积核，就生成了4张新特征图，记为(14,14,4)
model.add( tf.keras.layers.Conv2D(4, (3,3), input_shape=(28,28,1), activation = 'relu', padding = 'same') )
model.output_shape

(None, 14, 14, 4)

In [9]:
# 池化层2输出：同理，得(7,7,4)
model.add( tf.keras.layers.MaxPooling2D() )
model.output_shape

(None, 7, 7, 4)

In [10]:
# （普通）展平层：7*7*4 = 196 不管这张图已被转换成“多小多厚(7,7,4)”，都完全拆回成一个一维张量！
model.add( tf.keras.layers.Flatten() )
model.output_shape

(None, 196)

In [11]:
# 全连接层：设定64个神经元
model.add( tf.keras.layers.Dense(64) )
model.output_shape

(None, 64)

In [12]:
# 输出层：10分类，即10个神经元
model.add( tf.keras.layers.Dense(10) )
model.output_shape

(None, 10)